This first function adds random background noise from the musan data set in distored random intervals into the positive class and i changed directories to run on my negative data as well

In [ ]:
import os
import random
import torch
import torchaudio
import numpy as np
import librosa
import soundfile as sf

TARGET_SAMPLE_RATE = 16000
MAX_SECONDS = 15
MAX_SAMPLES = TARGET_SAMPLE_RATE * MAX_SECONDS

# Random noise dataset
def load_musan_noises(musan_dir):
    noise_files = []
    for subdir in ['noise', 'music', 'speech']:
        subpath = os.path.join(musan_dir, subdir)
        if os.path.exists(subpath):
            for root, _, files in os.walk(subpath):
                for file in files:
                    if file.endswith('.wav'):
                        noise_files.append(os.path.join(root, file))
    return noise_files

def add_background_noise(signal, sample_rate, noise_files, snr_db_range=(0, 15)):
    if not noise_files:
        return signal
    
    noise_path = random.choice(noise_files)
    noise, noise_sr = torchaudio.load(noise_path)
    
    if noise_sr != sample_rate:
        noise = torchaudio.transforms.Resample(noise_sr, sample_rate)(noise)
    
    sig_len = signal.shape[1]
    noise_len = noise.shape[1]

    if noise_len > sig_len:
        start = random.randint(0, noise_len - sig_len)
        noise = noise[:, start:start + sig_len]
    elif noise_len < sig_len:
        pad_left = random.randint(0, sig_len - noise_len)
        pad_right = sig_len - noise_len - pad_left
        noise = torch.nn.functional.pad(noise, (pad_left, pad_right))
    
    # Compute SNR
    snr_db = random.uniform(*snr_db_range)
    signal_power = torch.mean(signal ** 2)
    noise_power = torch.mean(noise ** 2)
    factor = torch.sqrt(signal_power / (noise_power * 10 ** (snr_db / 10)))
    noise.mul_(factor)  # in-place
    signal.add_(noise)  # in-place
    signal.div_(torch.max(torch.abs(signal)))  # normalize in-place
    return signal

def time_stretch(signal, rate):
    signal_np = signal.numpy().flatten()
    stretched = librosa.effects.time_stretch(signal_np, rate=rate)
    return torch.tensor(stretched, dtype=torch.float32).unsqueeze(0)

def apply_manual_gain(signal, gain_db):
    gain_linear = 10 ** (gain_db / 20)
    signal.mul_(gain_linear)  # in-place
    return signal

def augment_audio(audio_path, musan_noises, output_dir):
    signal, orig_sr = torchaudio.load(audio_path)
    
    # Resample if needed
    if orig_sr != TARGET_SAMPLE_RATE:
        signal = torchaudio.transforms.Resample(orig_sr, TARGET_SAMPLE_RATE)(signal)
    
    # Trim long audio
    if signal.shape[1] > MAX_SAMPLES:
        signal = signal[:, :MAX_SAMPLES]

    # Augmentations
    if random.random() < 0.5:
        signal = add_background_noise(signal, TARGET_SAMPLE_RATE, musan_noises)

    if random.random() < 0.5:
        pitch_shift = random.uniform(-4, 4)
        signal = torchaudio.functional.pitch_shift(signal, TARGET_SAMPLE_RATE, n_steps=pitch_shift)
    
    if random.random() < 0.5:
        rate = random.uniform(0.8, 1.2)
        signal = time_stretch(signal, rate)
    
    if random.random() < 0.5:
        gain_db = random.uniform(-6, 6)
        signal = apply_manual_gain(signal, gain_db)
    
    if random.random() < 0.3:
        noise = torch.normal(0, random.uniform(0.001, 0.015), signal.shape)
        signal.add_(noise)
        signal.div_(torch.max(torch.abs(signal)))  # normalize

    # Save augmented file
    base_name = os.path.basename(audio_path)
    output_path = os.path.join(output_dir, f"aug_{base_name}")
    sf.write(output_path, signal.numpy().flatten(), TARGET_SAMPLE_RATE)

    # Free memory
    del signal
    torch.cuda.empty_cache()
    
    return output_path

def main(input_dir, musan_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    musan_noises = load_musan_noises(musan_dir)
    if not musan_noises:
        print("Warning: No MUSAN noise files found. Skipping background noise augmentation.")
    
    # Recursively gather all WAV files
    wav_files = []
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
    
    print(f"Found {len(wav_files)} WAV files to augment.")

    for idx, audio_path in enumerate(wav_files, 1):
        try:
            augmented_path = augment_audio(audio_path, musan_noises, output_dir)
            print(f"[{idx}/{len(wav_files)}] Augmented and saved: {augmented_path}")
        except Exception as e:
            print(f"[{idx}/{len(wav_files)}] Error processing {audio_path}: {e}")

if __name__ == "__main__":
    musan_dir = "/Users/sethwright/Downloads/musan"
    input_dir = "/Users/sethwright/Documents/audio-model/data/sheila"
    output_dir = "/Users/sethwright/Documents/audio-model/data/Training_POS"
    main(input_dir, musan_dir, output_dir)


This function is to split a long background noise mp3 i found into wav and into 1 second intervals

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np
# Split background noise data set from long mp3 for negative class
def convert_and_split_mp3s(
    input_dir = "/Users/sethwright/Downloads/archive/_background_noise_"    ,
    output_dir = "/Users/sethwright/Documents/audio-model/data/background_data" ,
    sample_rate=16000,
    clip_duration=1,
    overlap_duration=0.25
):

    os.makedirs(output_dir, exist_ok=True)
    
    # Convert durations to samples
    clip_samples = int(sample_rate * clip_duration)
    overlap_samples = int(sample_rate * overlap_duration)
    step_size = clip_samples - overlap_samples  # how far we move forward each chunk

    # Walk through all files in the input directory
    for root, _, files in os.walk(input_dir):
        for fname in files:
            if not fname.lower().endswith(".wav"):
                continue

            in_path = os.path.join(root, fname)
            base_name = os.path.splitext(fname)[0]

            try:
                # -------------------------------------------------------
                # 1️⃣ Load audio (mono, 16 kHz)
                # -------------------------------------------------------
                y, sr = librosa.load(in_path, sr=sample_rate, mono=True)
                total_samples = len(y)

                # -------------------------------------------------------
                # 2️⃣ Compute how many clips will be produced
                # -------------------------------------------------------
                num_chunks = int(np.ceil((total_samples - overlap_samples) / step_size))

                # -------------------------------------------------------
                # 3️⃣ Split into overlapping chunks
                # -------------------------------------------------------
                for i in range(num_chunks):
                    start = i * step_size
                    end = start + clip_samples
                    clip = y[start:end]

                    # Pad the last clip with zeros if too short
                    if len(clip) < clip_samples:
                        clip = np.pad(clip, (0, clip_samples - len(clip)))

                    # Output filename: base_chunk001.wav
                    out_name = f"{base_name}_chunk{i+1:03d}.wav"
                    out_path = os.path.join(output_dir, out_name)

                    # ---------------------------------------------------
                    # 4️⃣ Save WAV file (float32 → 16-bit PCM)
                    # ---------------------------------------------------
                    sf.write(out_path, clip, sample_rate, subtype='PCM_16')

                print(f"✅ {fname}: {num_chunks} clips created")

            except Exception as e:
                print(f"❌ Error processing {fname}: {e}")

    print("\n🎉 All files converted and split successfully!")


In [ ]:
convert_and_split_mp3s()

In [ ]:
import os
import wave
import contextlib
# Find average Legnth to use for your model specs 
# Path to your directory containing audio files
audio_dir = "/Users/sethwright/Documents/audio-model/data/sheila"

durations = []

# Loop through all files in the directory
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):  # Change this if your files have a different extension
        file_path = os.path.join(audio_dir, filename)
        with contextlib.closing(wave.open(file_path, 'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)
            durations.append(duration)

if durations:
    smallest = min(durations)
    largest = max(durations)
    average = sum(durations) / len(durations)
    print(f"Smallest duration: {smallest:.2f} seconds")
    print(f"Largest duration: {largest:.2f} seconds")
    print(f"Average duration: {average:.2f} seconds")
else:
    print("No audio files found in the directory.")

# Delete files too far out of range
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):  # only process WAV files
        file_path = os.path.join(audio_dir, filename)
        with contextlib.closing(wave.open(file_path, 'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            duration = frames / float(rate)
            if duration < .6:
                print(f"Deleting {filename} (duration: {duration:.2f}s)")
                os.remove(file_path)